In [1]:
import transformer_lens.utils as utils
from transformer_lens import HookedTransformer, HookedTransformerConfig
# from mech_interp_othello_utils import OthelloBoardState
import einops
import torch
from tqdm import tqdm
import numpy as np
from fancy_einsum import einsum
import chess
import numpy as np
import csv
import chess_utils

device = "cuda"
device = "cpu"
device = "mps"

n_layers = 16
n_heads = 8
MODEL_DIR = "models/"
DATA_DIR = "data/"
cfg = HookedTransformerConfig(
    n_layers = n_layers,
    d_model = 512,
    d_head = 64,
    n_heads = n_heads,
    d_mlp = 2048,
    d_vocab = 32,
    n_ctx = 1023,
    act_fn="gelu",
    normalization_type="LNPre"
)
model = HookedTransformer(cfg)
model.load_state_dict(torch.load(f'{MODEL_DIR}tf_lens_16.pth'))
model.to(device)

/Users/akarvonen/anaconda3/envs/othello/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Moving model to device:  mps


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-15): 16 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [2]:
board_seqs_int = torch.tensor(np.load(f"{DATA_DIR}train_board_seqs_int.npy")).long()
print(board_seqs_int.shape)
dots_indices = torch.tensor(np.load(f"{DATA_DIR}train_dots_indices.npy")).long()
# state_stack = torch.tensor(np.load("state_stacks_5k.npy")).long()
print(dots_indices.shape)
# print(state_stack.shape)

board_seqs_string = []

with open(f"{DATA_DIR}train_board_seqs_string.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        board_seqs_string.append(row[0])
print(len(board_seqs_string), len(board_seqs_string[0]))
# print(board_seqs_string[0])


torch.Size([32965, 680])
torch.Size([32965, 61])
32965 680


In [3]:
# print(board_seqs_string[0])
custom_function = chess_utils.board_to_piece_color_state
print(chess_utils.create_state_stack(board_seqs_string[0], custom_function).shape)

state_stack = torch.tensor(chess_utils.create_state_stack(board_seqs_string[0], custom_function)).long()
print(state_stack.shape)

state_stacks = chess_utils.create_state_stacks(board_seqs_string[:50], custom_function)
print(state_stacks.shape)

(680, 8, 8)
torch.Size([680, 8, 8])
torch.Size([50, 680, 8, 8])


In [4]:
layer = 12
batch_size = 2
lr = 2e-4
wd = 0.01
pos_start = 0
# pos_end = model.cfg.n_ctx - 5
input_length = 680
pos_end = input_length - 0
length = pos_end - pos_start
options = 3
rows = 8
cols = 8
num_epochs = 2
num_games = 20000
x = 0
y = 2
probe_name = "main_chess_linear_probe"
# The first mode is blank or not, the second mode is next or prev GIVEN that it is not blank
modes = 1
alternating = torch.tensor([1 if i%2 == 0 else -1 for i in range(length)], device=device)


state_stack_one_hot = chess_utils.state_stack_to_one_hot(modes, rows, cols, options, device, state_stacks)
print(state_stack_one_hot.shape)
print((state_stack_one_hot[:, 1, 170, 4:9, 2:5]))
print((state_stacks[1, 170, 4:9, 2:5]))

torch.Size([1, 50, 680, 8, 8, 3])
tensor([[[[1, 0, 0],
          [0, 1, 0],
          [0, 1, 0]],

         [[0, 1, 0],
          [0, 1, 0],
          [1, 0, 0]],

         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],

         [[1, 0, 0],
          [1, 0, 0],
          [0, 1, 0]]]], device='mps:0')
tensor([[ 0, -1, -1],
        [-1, -1,  0],
        [ 0,  0,  0],
        [ 0,  0, -1]])


In [5]:
linear_probe = torch.randn(
    modes, model.cfg.d_model, rows, cols, options, requires_grad=False, device=device
)/np.sqrt(model.cfg.d_model)
linear_probe.requires_grad = True
print(linear_probe.shape)
optimiser = torch.optim.AdamW([linear_probe], lr=lr, betas=(0.9, 0.99), weight_decay=wd)

print(dots_indices.shape)
# mask = dots_indices < 245
# dots_indices = dots_indices[mask]

# print(dots_indices.shape)

lr = 3e-4
max_lr = 3e-4
min_lr = lr / 10
max_iters = num_games * num_epochs
decay_lr = True

def get_lr(current_iter: int, max_iters: int, lr: float, min_lr: float) -> float:
    """
    Calculate the learning rate using linear decay.

    Args:
    - current_iter (int): The current iteration.
    - max_iters (int): The total number of iterations for decay.
    - lr (float): The initial learning rate.
    - min_lr (float): The minimum learning rate after decay.

    Returns:
    - float: The calculated learning rate.
    """
    # Ensure current_iter does not exceed max_iters
    current_iter = min(current_iter, max_iters)

    # Calculate the linearly decayed learning rate
    decayed_lr = lr - (lr - min_lr) * (current_iter / max_iters)

    return decayed_lr
current_iter = 0
for epoch in range(num_epochs):
    full_train_indices = torch.randperm(num_games)
    for i in tqdm(range(0, num_games, batch_size)):

        lr = get_lr(current_iter, max_iters, max_lr, min_lr) if decay_lr else lr
        for param_group in optimiser.param_groups:
            param_group['lr'] = lr
        
        indices = full_train_indices[i:i+batch_size]
        list_of_indices = indices.tolist() # For indexing into the board_seqs_string list of strings
        # print(list_of_indices)
        games_int = board_seqs_int[indices]
        games_int = games_int[:, :input_length]
        # print(games_int.shape)
        games_str = [board_seqs_string[idx] for idx in list_of_indices]
        games_str = [s[:input_length] for s in games_str]
        games_dots = dots_indices[indices]
        games_dots = games_dots[:, 5:]
        # print(games_dots.shape)
        state_stack = chess_utils.create_state_stacks(games_str, custom_function)
        # state_stack = state_stack[:, pos_start:pos_end, :, :]
        # print(state_stack.shape)
        # Initialize a list to hold the indexed state stacks
        indexed_state_stacks = []

        for batch_idx in range(state_stack.size(0)):
            # Get the indices for the current batch
            dots_indices_for_batch = games_dots[batch_idx]

            # Index the state_stack for the current batch
            indexed_state_stack = state_stack[batch_idx, dots_indices_for_batch, :, :]

            # Append the result to the list
            indexed_state_stacks.append(indexed_state_stack)

        # Stack the indexed state stacks along the first dimension
        # This results in a tensor of shape [2, 61, 8, 8] (assuming all batches have 61 indices)
        state_stack = torch.stack(indexed_state_stacks)
        # print("after indexing state stack shape", state_stack.shape)

        state_stack_one_hot = chess_utils.state_stack_to_one_hot(modes, rows, cols, options, device, state_stack).to(device)
        with torch.inference_mode():
            _, cache = model.run_with_cache(games_int.to(device)[:, :-1], return_type=None)
            resid_post = cache["resid_post", layer][:, :]
        # Initialize a list to hold the indexed state stacks
        indexed_resid_posts = []

        for batch_idx in range(games_dots.size(0)):
            # Get the indices for the current batch
            dots_indices_for_batch = games_dots[batch_idx]

            # Index the state_stack for the current batch
            indexed_resid_post = resid_post[batch_idx, dots_indices_for_batch]

            # Append the result to the list
            indexed_resid_posts.append(indexed_resid_post)

        # Stack the indexed state stacks along the first dimension
        # This results in a tensor of shape [2, 61, 8, 8] (assuming all batches have 61 indices)
        resid_post = torch.stack(indexed_resid_posts)
        # print("Resid post", resid_post.shape)
        probe_out = einsum(
            "batch pos d_model, modes d_model rows cols options -> modes batch pos rows cols options",
            resid_post,
            linear_probe,
        )
        # print(probe_out.shape)

        # acc_blank = (probe_out[0].argmax(-1) == state_stack_one_hot[0].argmax(-1)).float().mean()
        # acc_color = ((probe_out[1].argmax(-1) == state_stack_one_hot[1].argmax(-1)) * state_stack_one_hot[1].sum(-1)).float().sum()/(state_stack_one_hot[1]).float().sum()

        

        probe_log_probs = probe_out.log_softmax(-1)
        probe_correct_log_probs = einops.reduce(
            probe_log_probs * state_stack_one_hot,
            "modes batch pos rows cols options -> modes pos rows cols",
            "mean"
        ) * options # Multiply to correct for the mean over options
        # loss_even = -probe_correct_log_probs[0, 0::2].mean(0).sum() # note that "even" means odd in the game framing, since we offset by 5 moves lol
        # loss_odd = -probe_correct_log_probs[1, 1::2].mean(0).sum()
        loss_all = -probe_correct_log_probs[0, :].mean(0).sum()

        # if i % 1000 == 0:
        #     print(f"epoch {epoch}, batch {i}, acc_blank {acc_blank}, acc_color {acc_color}, loss_even {loss_even}, loss_odd {loss_odd}, loss_all {loss_all}")
        if i % 100 == 0:
            print(f"epoch {epoch}, batch {i}, loss_all {loss_all}, lr {lr}")
        # loss = loss_even + loss_odd + loss_all
        loss = loss_all
        loss.backward() # it's important to do a single backward pass for mysterious PyTorch reasons, so we add up the losses - it's per mode and per square.

        optimiser.step()
        optimiser.zero_grad()
        current_iter += batch_size
torch.save(linear_probe, f"{probe_name}.pth")

torch.Size([1, 512, 8, 8, 3])
torch.Size([32965, 61])


  0%|          | 1/10000 [00:00<57:11,  2.91it/s]

epoch 0, batch 0, loss_all 73.68692016601562, lr 0.0003


  0%|          | 24/10000 [00:02<19:08,  8.68it/s]


KeyboardInterrupt: 